In [63]:
import xarray as xr
import numpy as np
import pandas as pd

val_df = pd.read_parquet('../data/training_data/valdf_100km_random_reshaped.pq')
val_df2021 = pd.read_parquet('../data/training_data/df_100km_random_reshaped_2021.pq')
train_df = pd.read_parquet('../data/training_data/traindf_100km_random_reshaped.pq')

In [64]:
def add_xco2(df):
    day_year = df[['day_of_year', 'year']].astype(int)
    df['time'] = pd.to_datetime(day_year['day_of_year'].astype(str) + day_year['year'].astype(str), format='%j%Y')

    
    selector = df[['lat', 'time']].to_xarray()
    xco2mbl = xr.open_dataarray('../data/atmco2/xco2mbl-timeP7D_1D-lat25km.nc')
    matched_xco2 = xco2mbl.sel(**selector, method='nearest').to_series()
    df['xco2'] = matched_xco2

    return df 

In [ ]:
# find all segments in vald_df2021 which have one year value whihc is nan and remove them from the dataframe
# for val_df and train_df this was already done in make_augmented_set.ipynb
segments = val_df2021.groupby('segment').filter(
    lambda x: x['year'].isna().any()
    ).index.get_level_values('segment').unique()
print(f"Number of segments with one year value which is nan: {len(segments)}")
# segment is aprt of he index
val_df2021 = val_df2021.reset_index()
val_df2021 = val_df2021[~val_df2021['segment'].isin(segments)]
val_df2021 = val_df2021.set_index(['segment', 'bin'])


Number of segments with one year value which is nan: 2317


In [76]:
# check if year and day_of_year contain nan values
print(f"Number of nan values in year: {val_df2021['year'].isna().sum()}")
print(f"Number of nan values in day_of_year: {val_df2021['day_of_year'].isna().sum()}")
print(f"Number of nan values in year: {train_df['year'].isna().sum()}")
print(f"Number of nan values in day_of_year: {train_df['day_of_year'].isna().sum()}")
print(f"Number of nan values in year: {val_df['year'].isna().sum()}")
print(f"Number of nan values in day_of_year: {val_df['day_of_year'].isna().sum()}")

Number of nan values in year: 0
Number of nan values in day_of_year: 0
Number of nan values in year: 0
Number of nan values in day_of_year: 0
Number of nan values in year: 0
Number of nan values in day_of_year: 0


In [71]:
val_df = add_xco2(val_df)
vald_df2021 = add_xco2(val_df2021)
train_df = add_xco2(train_df)

In [72]:
val_df.index.is_unique, vald_df2021.index.is_unique, train_df.index.is_unique

(True, True, True)

In [73]:
# check for nans in xco2 column
val_df['xco2'].isna().sum(), val_df2021['xco2'].isna().sum(), train_df['xco2'].isna().sum()

(np.int64(0), np.int64(0), np.int64(0))

In [74]:
# sve the dataframes to parquet files
val_df.to_parquet('../data/training_data/valdf_100km_xco2.pq')
val_df2021.to_parquet('../data/training_data/df_100km_xco2_2021.pq')
train_df.to_parquet('../data/training_data/traindf_100km_xco2.pq')